In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Scrapping with selenium

In [127]:
def data_adq(link, year, df_name, jornada, dic):
    #Scrape data from match and adds it to df
    driver2 = webdriver.Safari()
    driver2.get(link)
    time.sleep(1)
    accept = driver2.find_elements(By.XPATH, '//*[@id="didomi-notice-agree-button"]')
    accept[0].click()
    stats = driver2.find_elements(By.TAG_NAME, 'a')
    stats[10].click()
    line1 = [1,jornada]
    line2=[1,jornada]
    table = driver2.find_elements(By.TAG_NAME, 'section')
    diayhora = table[0]
    a=diayhora.text.split()
    dia = a[5][0]
    line1.append(dia)
    line2.append(dia)
    fecha = a[5][1:6] + f'/{year}'
    line1.append(fecha)
    line2.append(fecha)
    hora = a[6][0:5]
    line1.append(hora)
    line2.append(hora)
    gion = a.index('-')
    goles_casa = int(a[gion-1][-1])
    goles_visita = int(a[gion+1][0])
    line1.append(goles_casa)
    line1.append(goles_visita)
    line2.append(goles_visita)
    line2.append(goles_casa)
    equipos = driver2.find_elements(By.CSS_SELECTOR, 'h2.team-name')
    equipo_casa = equipos[0].text
    equipo_visita = equipos[1].text
    line1.append(equipo_casa)
    line1.append(equipo_visita)
    line2.append(equipo_visita)
    line2.append(equipo_casa)
    numbers = driver2.find_elements(By.CSS_SELECTOR, 'div.voc-stat-item')
    posesion = driver2.find_elements(By.CSS_SELECTOR, 'span.number')
    home_pos = posesion[0].text
    away_pos = posesion[1].text
    line1.append(home_pos)
    line2.append(away_pos)
    a = [i.text for i in numbers]
    a  = [s.replace("\n", "") for s in a]
    a = a[1:17]
    for stat in a:
        stat = stat.split()
        if len(stat) == 3:
            line1.append(stat[0])
            line2.append(stat[2])
        if len(stat) == 4:
            line1.append(stat[0])
            line2.append(stat[3])
        if len(stat) == 5:
            line1.append(stat[0])
            line2.append(stat[4])
    line1.append('Si')
    line2.append('No')
    line1.append('19/20')
    line2.append('19/20')
    line1.append(dic[equipo_casa])
    line2.append(dic[equipo_visita])
    df.loc[len(df)] = line1
    df.loc[len(df)] = line2
    driver2.close()

In [27]:
cols=['Division', 'Jornada', 'Dia','Fecha','Hora','GF','GA','Equipo','Oponente','Posesion', 'Remates a Puerta', 'Remates Fuera', 'Remates de Palo', 'Asistencias', 'Asistencias de gol', 'Faltas Cometidas', 'Faltas Recibidas', 'Tarjetas Amarillas', 'Tarjetas Rojas', 'Pases Correctos', 'Pases Fallados', 'Fueras de Juego', 'Paradas', 'Corners', 'Penaltis a Favor', 'Penaltis en Contra', 'Casa', 'Temporada', 'pos']
df = pd.DataFrame(columns=cols)
#Empty Df

In [142]:
#Scrapping data before the new year
for i in range(1,19):
    web = f'https://www.elcorreo.com/deportes/futbol/liga-primera/2019-2020/jornada-{i}/clasificacion-resultados-sd.html'
    driver = webdriver.Safari()
    driver.get(web)
    accept = driver.find_elements(By.XPATH, '//*[@id="didomi-notice-agree-button"]')
    accept[0].click()
    table = driver.find_elements(By.CSS_SELECTOR, 'table.voc-resultados-list')
    anchor_elements = table[0].find_elements(By.TAG_NAME, 'a')
    href_links = [anchor.get_attribute('href') for anchor in anchor_elements]
    links = []
    table2 = driver.find_elements(By.CSS_SELECTOR, 'table.voc-clasificacion-list')
    rows = table2[0].find_elements(By.CSS_SELECTOR, 'tr.voc-clasificacion-item')
    dic = {}
    for row in rows:
        cosa = row.find_element(By.CSS_SELECTOR, 'td.equipo')
        equipo = cosa.find_element(By.CSS_SELECTOR,'a.nombre-equipo').text
        pos = cosa.find_element(By.CSS_SELECTOR,'span.posicion-tabla').text
        dic[equipo] = int(pos)
    for cosa in href_links:
        if 'jornada' not in cosa:
            links.append(cosa)
    driver.close()
    for link in links:
        data_adq(link,2019,df, i, dic)
    
#Scrapping data after the new year
for i in range(19,39):
    web = f'https://www.elcorreo.com/deportes/futbol/liga-primera/2019-2020/jornada-{i}/clasificacion-resultados-sd.html'
    driver = webdriver.Safari()
    driver.get(web)
    accept = driver.find_elements(By.XPATH, '//*[@id="didomi-notice-agree-button"]')
    accept[0].click()
    table = driver.find_elements(By.CSS_SELECTOR, 'table.voc-resultados-list')
    anchor_elements = table[0].find_elements(By.TAG_NAME, 'a')
    href_links = [anchor.get_attribute('href') for anchor in anchor_elements]
    links = []
    table2 = driver.find_elements(By.CSS_SELECTOR, 'table.voc-clasificacion-list')
    rows = table2[0].find_elements(By.CSS_SELECTOR, 'tr.voc-clasificacion-item')
    dic = {}
    for row in rows:
        cosa = row.find_element(By.CSS_SELECTOR, 'td.equipo')
        equipo = cosa.find_element(By.CSS_SELECTOR,'a.nombre-equipo').text
        pos = cosa.find_element(By.CSS_SELECTOR,'span.posicion-tabla').text
        dic[equipo] = int(pos)
    for cosa in href_links:
        if 'jornada' not in cosa:
            links.append(cosa)
    driver.close()
    for link in links:
        data_adq(link,2020,df, i, dic)


# Tranforming and prepping Data

In [ ]:
#W/L/D
def compare_values(row):
    if row['GF'] > row['GA']:
        return 'W'
    elif row['GF'] < row['GA']:
        return 'L'
    else:
        return 'D'
df['Resultado'] = df.apply(compare_values, axis=1)

In [ ]:
df['Presición de Pases'] = df['Pases Correctos']/(df['Pases Correctos'] + df['Pases Fallados'])*100

In [ ]:
df['Remates Totales'] = df['Remates a Puerta'] + df['Remates Fuera'] + df['Remates de Palo']

In [ ]:
import numpy as np
df['Conversión Tiros Totales'] = np.where(df['Remates Totales'].isna() | (df['Remates Totales'] == 0), 
                                          0, 
                                          (df['GF'] / df['Remates Totales']) * 100)

In [ ]:
df['Conversión Tiros Puerta'] = np.where(df['Remates a Puerta'].isna() | (df['Remates a Puerta'] == 0), 
                                          0, 
                                          (df['GF'] / df['Remates a Puerta']) * 100)

In [ ]:
v = []
# To calculate ppoints per matchday
for i in range(len(df)):
    team = df.iloc[i]['Equipo']
    jornada = df.iloc[i]['Jornada']
    temp = df.iloc[i]['Temporada']
    # Filter the DataFrame to only include rows for the same team and earlier matchdays
    df_filtered = df[(df['Equipo'] == team) & (df['Jornada'] < jornada) & (df['Temporada'] == temp) ]
    
    # Calculate points based on the filtered DataFrame
    points = 0
    for result in df_filtered['Resultado']:
        if result == 'W':
            points += 3
        elif result == 'D':
            points += 1
    v.append(points)

In [ ]:
df['Puntos'] = v

In [ ]:
GD = []

for i in range(len(df)):
    team = df.iloc[i]['Equipo']
    jornada = df.iloc[i]['Jornada']
    temp = df.iloc[i]['Temporada']
    # Filter the DataFrame to only include rows for the same team and earlier matchdays
    df_filtered = df[(df['Equipo'] == team) & (df['Jornada'] < jornada) & (df['Temporada'] == temp)]
    
    # Calculate points based on the filtered DataFrame
    GF = df_filtered['GF'].sum()
    GA = df_filtered['GA'].sum()
    GD.append(GF-GA)
df['GD'] =GD

In [ ]:
df['Posesion'] = df['Posesion'].str.rstrip('%').astype(float)

In [ ]:
def av(col,df):
    stat = []
    for i in range(len(df)):
        team = df.iloc[i]['Equipo']
        jornada = df.iloc[i]['Jornada']
        temp = df.iloc[i]['Temporada']
        # Filter the DataFrame to only include rows for the same team and earlier matchdays
        df_filtered = df[(df['Equipo'] == team) & (df['Jornada'] < jornada) & (df['Temporada'] == temp)]

        # Calculate points based on the filtered DataFrame
        summ = df_filtered[col].sum()
        stat.append(round(summ/jornada,2))
        name = col + '/game'
    df[name] = stat

In [ ]:
cosa = ['GF', 'GA', 'Posesion', 'Remates a Puerta', 'Remates Fuera',
       'Remates de Palo', 'Asistencias', 'Asistencias de gol',
       'Faltas Cometidas', 'Faltas Recibidas', 'Tarjetas Amarillas',
       'Tarjetas Rojas', 'Pases Correctos', 'Pases Fallados',
       'Fueras de Juego', 'Paradas', 'Corners', 'Penaltis a Favor',
       'Penaltis en Contra',
       'Presición de Pases', 'Remates Totales', 'Conversión Tiros Totales',
       'Conversión Tiros Puerta']
for stat in cosa:
    av(stat, df)

**Saving to CSV**

In [145]:
df.to_csv('full_1_202324.csv', index = False)